In [66]:
import numpy as np
from matplotlib import pyplot as plt
import time
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader


In [67]:
class Rect():
    def __init__(self, xs, ys, xe, ye):
        self.xs = xs # x start
        self.ys = ys # y start
        self.xe = xe # x end
        self.ye = ye # y end

    def __array__(self):
        return np.array([self.xs, self.ys, self.xe, self.ye])


In [68]:
np.array([(*np.array([1,2,3]), *np.array([4,5,6]),1)])

array([[1, 2, 3, 4, 5, 6, 1]])

In [133]:
def describe_features(shape):
    rect2 = []
    rect3 = []
    rect4 = []
    height, width = shape
    cnt = 0
    for i in range(height+1):
        for j in range(width+1):
            for k in range(1, height+1):
                for l in range(1, width+1):
                    cnt += 1
                    # @ 2 features
                    # Horizontal [-][+]
                    if i + k <= height and j + 2 * l <= width:
                        # start positive i, start positive j, end positive i, end positive j
                        rect_p = Rect(i, j + l, i + k, j + 2 * l)
                        # start negative i, start negative j, end negative i, end negative j
                        rect_n = Rect(i, j, i + k, j + l)
                        rect2.append(
                            # (np.array(rect_p), np.array(rect_n)))
                            (*np.array(rect_p), *np.array(rect_n), 0))

                    # Vertical #+
                        # -
                    if i + 2 * k <= height and j + l <= width:
                        rect_p = Rect(i, j, i + k, j + l)
                        rect_n = Rect(i + k, j, i + 2 * k, j + l)
                        rect2.append(
                            (*np.array(rect_p), *np.array(rect_n), 1))

                    # @ 3 features
                    # Horizontal [-][+][-]
                    if i + k <= height and j + 3 * l <= width:
                        rect_n1 = Rect(i, j, i + k, j + l)
                        rect_p = Rect(i, j + l, i + k, j + 2 * l)
                        rect_n2 = Rect(i, j + 2 * l, i + k, j + 3 * l)
                        rect3.append(
                            (np.array(rect_n1), np.array(rect_p), np.array(rect_n2)))

                    # Vertical #-
                        # +
                        # -
                    if i + 3 * k <= height and j + l <= width:
                        rect_n1 = Rect(i, j, i + k, j + l)
                        rect_p = Rect(i + k, j, i + 2 * k, j + l)
                        rect_n2 = Rect(i + 2 * k, j, i + 3 * k, j + l)
                        rect3.append((np.array(rect_n1),np.array(rect_p),np.array(rect_n2)))

                    # @ 4 features
                    # [-][+]
                    # [+][-]
                    if i + 2 * k <= height and j + 2 * l <= width:
                        rect_n1 = Rect(i, j, i + k, j + l)
                        rect_p = Rect(i, j + l, i + k, j + 2 * l)
                        rect_p2 = Rect(i + k, j, i + 2 * k, j + l)
                        rect_n2 = Rect(i + k, j + l, i + 2 * k, j + 2 * l)
                        rect4.append((np.array(rect_n1),np.array(rect_p),np.array(rect_p2),np.array(rect_n2)))
    return np.array(rect2), np.array(rect3), np.array(rect4), cnt


In [150]:
%time
f2, f3, f4, c= describe_features((24,24))

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 6.2 µs


In [151]:
print(f2.shape)
print(f3.shape)
print(f4.shape)
print(f2.shape[0] + f3.shape[0] + f4.shape[0])
print(c)

(86400, 9)
(55200, 3, 4)
(20736, 4, 4)
162336
360000


# Comparison

In [152]:
fkak = np.load('/home/akram/CMP4/GP/Try/ft.npy')
# fkak = np.load('/home/akram/CMP4/GP/Try/f5.npy')

In [153]:
fkak.shape

(162336, 5)

In [154]:
f22 = f2[f2[:, -1] == 0]
f22 = np.stack((np.zeros(len(f22)), f22[:,5],f22[:,0], f22[:,3] - f22[:,5], f22[:,2] - f22[:,0]), axis=1)
fk2 = fkak[fkak[:,0] == 0]
f22 = f22[np.lexsort((f22[:,4],f22[:,3],f22[:,2], f22[:, 1], f22[:, 0]))]
fk2 = fk2[np.lexsort((fk2[:,4],fk2[:,3],fk2[:,2], fk2[:, 1], fk2[:, 0]))]
print(np.sum(f22 == fk2), f22.size)
print(f22.shape)
print(fk2.shape)


216000 216000
(43200, 5)
(43200, 5)


In [147]:
for i in range(5):
    print(np.sum(fk2[:,i]==f22[:,i]))

90
90
90
90
90


# Features Again

In [ ]:
def featuresAgain(shape):
    height, width = shape
    rect2 = []
    rect3 = []
    rect4 = []
    

In [79]:
for i in range(5):
    l = fkak[fkak[:,0] == i]
    print(l.shape)

(43200, 5)
(43200, 5)
(27600, 5)
(27600, 5)
(20736, 5)
